In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI()

template = PromptTemplate.from_template("What is the advantage of {frontend_stack}?")

prompt = template.format(frontend_stack="Svelte")

answer = chat.predict(prompt)

answer

'There are several advantages of using Svelte as a JavaScript framework:\n\n1. Performance: Svelte compiles the code during the build process, which means it generates highly optimized and efficient code. This results in faster loading times and better overall performance compared to other frameworks.\n\n2. Small bundle size: Svelte generates small bundle sizes by eliminating the need for a runtime library. This reduces the amount of code that needs to be downloaded and executed by the browser, resulting in faster page loads.\n\n3. Framework-agnostic: Unlike many other frameworks, Svelte is not tied to a specific ecosystem or toolset. It can be easily integrated into existing projects or used with other libraries and frameworks.\n\n4. Easy learning curve: Svelte has a simpler and more intuitive syntax compared to other frameworks like React or Angular. This makes it easier for developers to learn and start building applications quickly.\n\n5. Reactive updates: Svelte uses a reactive ap

In [30]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a frontend expert. Also, you only reply in {language}"),
        ("ai", "{tone}"),
        ("human", "What is the advantage of {frontend_stack}? Also, What's your name?"),
    ]
)

prompt = template.format_messages(
    language="Korean", tone="안녕하세요!", frontend_stack="Next.js"
)

chat.predict_messages(prompt)

AIMessage(content='안녕하세요! Next.js의 장점에는 몇 가지가 있습니다. \n\n첫째, 서버 사이드 렌더링(SSR)을 기본으로 제공하기 때문에 초기 로딩 속도가 빠릅니다. 이는 검색 엔진 최적화(SEO)에 유리하며, 사용자 경험을 향상시킵니다.\n\n둘째, 코드 스플리팅(Code splitting)을 자동으로 처리하여 필요한 컴포넌트들만 로드하므로 초기 번들 크기가 작아지고 성능이 향상됩니다.\n\n셋째, Hot Module Replacement(HMR)을 지원하여 개발 중인 코드를 수정할 때 즉시 반영되어 개발 생산성을 높일 수 있습니다.\n\n넷째, 강력한 라우팅 시스템을 제공하여 페이지 간 전환 및 동적 라우팅을 쉽게 구현할 수 있습니다.\n\n저는 AI 모델이기 때문에 이름을 갖지 않습니다. 그러나 여러분이 부르는 대로 대화를 이어나갈 수 있습니다! 무엇을 도와드릴까요?')

In [31]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        return list(map(str.strip, text.split(",")))


parser = CommaOutputParser()

parser.parse(" Hello, how,are,you ")

['Hello', 'how', 'are', 'you']

In [32]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else",
        ),
        ("human", "{question}"),
    ]
)

In [38]:
chain = template | chat | parser

chain.invoke(
    {"max_items": 5, "question": "Interesting things about frontend development"}
)

['html', 'css', 'javascript', 'responsive design', 'user experience']